# **Diabetic Retinopathy using CNNs**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset_path = "/content/drive/MyDrive/Dataset_DiabeticRetenopathy dataset/Diabetic Retinopathy Dataset CNN"

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

# Step 1: Load the data
def load_data(data_dir):
    images = []
    labels = []
    for label in os.listdir(data_dir):
        for image_file in os.listdir(os.path.join(data_dir, label)):
            image_path = os.path.join(data_dir, label, image_file)
            image = cv2.imread(image_path)
            image = cv2.resize(image, (128, 128))  # Resize the image if necessary
            images.append(image)
            labels.append(int(label))
    return np.array(images), np.array(labels)

# X, y = load_data(r"C:\Users\subba\Downloads\Dataset_DiabeticRetenopathy dataset\Dataset_DiabeticRetenopathy")
X, y = load_data(dataset_path)
X = X / 255.0  # Normalize the pixel values

**Dataset is imbalanced so causing Bias in the Model**

So the normal train-test splitting shouldn't be done.



```
# Step 2: Split the data into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2
```



To rectify this imbalance, you can use various techniques. One common approach is to balance the class distribution during training. Here's how you can do it:

1. **Data Augmentation:** Generate additional synthetic data for the minority class (diseased images) using techniques like rotation, flipping, scaling, etc.
2. **Resampling:** Either oversample the minority class (diseased images) or undersample the majority class (non-diseased images) to balance the class distribution.
3. **Class Weighting:** Assign higher weights to the minority class during training to make the model pay more attention to it.

Let's apply oversampling to balance the class distribution:

In [ ]:
from imblearn.over_sampling import RandomOverSampler

# Reshape the data for oversampling
X_reshaped = X.reshape(X.shape[0], -1)

# Apply Random Oversampling
oversampler = RandomOverSampler(random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X_reshaped, y)

# Reshape the data back to original shape
X_resampled = X_resampled.reshape(X_resampled.shape[0], 128, 128, 3)

# Split the resampled data into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2


In [ ]:
# Step 3: Design the CNN Architecture
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', padding='same', strides=2, input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same', strides=2),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same', strides=2),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Step 4: Save the best model
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
callbacks_list = [checkpoint]

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val), callbacks=callbacks_list)

# Step 5: Load the best model
best_model = load_model("best_model.h5")

# Step 6: Predict the test data
y_pred = best_model.predict(X_test)
y_pred = np.round(y_pred).flatten()

# Step 7: Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Epoch 1/20
14/14 [==============================] - ETA: 0s - loss: 0.6958 - accuracy: 0.5139
Epoch 1: val_accuracy improved from -inf to 0.46897, saving model to best_model.h5
14/14 [==============================] - 4s 128ms/step - loss: 0.6958 - accuracy: 0.5139 - val_loss: 0.7066 - val_accuracy: 0.4690
Epoch 2/20
 5/14 [=========>....................] - ETA: 0s - loss: 0.6846 - accuracy: 0.5875

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


 9/14 [==================>...........] - ETA: 0s - loss: 0.6945 - accuracy: 0.5382
Epoch 2: val_accuracy did not improve from 0.46897
14/14 [==============================] - 0s 19ms/step - loss: 0.6928 - accuracy: 0.5324 - val_loss: 0.6992 - val_accuracy: 0.4690
Epoch 3/20
11/14 [======================>.......] - ETA: 0s - loss: 0.6870 - accuracy: 0.5170
Epoch 3: val_accuracy improved from 0.46897 to 0.61379, saving model to best_model.h5
14/14 [==============================] - 0s 20ms/step - loss: 0.6856 - accuracy: 0.5301 - val_loss: 0.6857 - val_accuracy: 0.6138
Epoch 4/20
11/14 [======================>.......] - ETA: 0s - loss: 0.6709 - accuracy: 0.5938
Epoch 4: val_accuracy did not improve from 0.61379
14/14 [==============================] - 0s 17ms/step - loss: 0.6697 - accuracy: 0.5972 - val_loss: 0.6983 - val_accuracy: 0.5862
Epoch 5/20
10/14 [====================>.........] - ETA: 0s - loss: 0.6565 - accuracy: 0.6313
Epoch 5: val_accuracy did not improve from 0.61379
14/14 